Importing necessary libraries 

In [1]:
import numpy as np
import pandas as pd

Mounting the files

In [2]:
df_a=pd.read_csv("5_a.csv")
df_b=pd.read_csv("5_b.csv")
df_c=pd.read_csv("5_c.csv")
df_d=pd.read_csv("5_d.csv")

Probability Value --> Output Label 

In [3]:
df_a['y_pred']=df_a['proba'].apply(lambda x: 1 if x>=.5 else 0)
df_b['y_pred']=df_b['proba'].apply(lambda x: 1 if x>=.5 else 0)

Confusion matrix 

In [4]:
def confusion_matrix(data):
    count_tn=len(data[(data['y']==0) & (data['y_pred']==0)])
    count_tp=len(data[(data['y']==1) & (data['y_pred']==1)]) 
    count_fn=len(data[(data['y']==1) & (data['y_pred']==0)])
    count_fp=len(data[(data['y']==0) & (data['y_pred']==1)])
    return count_fn,count_fp,count_tn,count_tp

Calculating the F1 Score 


In [5]:
def f1_score(data):
    fn,fp,tn,tp=confusion_matrix(data)
    precision=tp/(tp+fp)                   
    recall=tp/(tp+fn)                    
    f1=2*((precision*recall)/(precision+recall))
    return f1

Calculating Accuracy Level

In [6]:
def accuracy(data):
    fn,fp,tn,tp=confusion_matrix(data)
    acc=((tp+tn)/(tp+fp+fn+tn))          
    return acc

The AUC Value 


In [7]:
def auc_score(data):
    tpr_array=[]
    fpr_array=[]
    sort= data.sort_values("proba",ascending=False) 
    for i in range(0,len(sort)):
        sort['y_pred']=np.where(sort['proba']>=sort.iloc[i]['proba'],1,0) 
        FN,FP,TN,TP=confusion_matrix(sort)    
        fpr_rate=FP/(TN+FP)
        tpr_rate=TP/(TP+FN)
        tpr_array.append(tpr_rate)
        fpr_array.append(fpr_rate)
    c=np.trapz(tpr_array, fpr_array)
    return c

**PERFORMANCE MATRIX FOR 5_a.csv**

Confusion matrix 5_a.csv

In [8]:
FN,FP,TN,TP=confusion_matrix(df_a)
print("False Negative --> ",FN)
print("False Positive --> ",FP)
print("True Negative  --> ",TN)
print("True Positive  --> ",TP)

False Negative -->  0
False Positive -->  100
True Negative  -->  0
True Positive  -->  10000


f1 Score 5_a.csv , Accuracy Value 5_a.csv , AUC Value 5_a.csv

In [9]:
f1=f1_score(df_a)
print("f1 Score:",f1)

print("------------------")

acc=accuracy(df_a)
print('Accuracy Value :',acc)


print("------------------")

auc=auc_score(df_a)
print('AUC Value :',auc)

f1 Score: 0.9950248756218906
------------------
Accuracy Value : 0.9900990099009901
------------------
AUC Value : 0.48829900000000004


Confusion matrix 5_b.csv

In [10]:
FN,FP,TN,TP=confusion_matrix(df_b)
print("False Negative --> ",FN)
print("False Positive --> ",FP)
print("True Negative  --> ",TN)
print("True Positive  --> ",TP)

False Negative -->  45
False Positive -->  239
True Negative  -->  9761
True Positive  -->  55


f1 Score 5_b.csv

In [11]:
f1=f1_score(df_b)
print("f1 Score :",f1)

print("-----------")

acc=accuracy(df_b)
print('Accuracy Level :',acc)

print("-----------")

auc=auc_score(df_b)
print("AUC Value :",auc)

f1 Score : 0.2791878172588833
-----------
Accuracy Level : 0.9718811881188119
-----------
AUC Value : 0.9377570000000001


Compute the best threshold of probability which gives lowest values of metric A for the given data 5_c.csv

In [12]:
def best_threshold(data):
    tester=0
    threshold=[]
    A=[]
    sorted= data.sort_values("prob",ascending=False) 
    for i in range(0,len(sorted)):
        if tester==(sorted.iloc[i]['prob']):
            continue
        tester=sorted.iloc[i]['prob'] 
        threshold.append(tester)
        sorted['y_pred']=np.where(sorted['prob']>=sorted.iloc[i]['prob'],1,0)
        FN,FP,TN,TP=confusion_matrix(sorted) 
        value=500*FN+100*FP
        A.append(value)  
    index=A.index(min(A)) 
    return threshold[index]

In [13]:
best=best_threshold(df_c)
print('Best Threshold Value -->',best)

Best Threshold Value --> 0.2300390278970873


Compute performance metrics(for regression) for the given data 5_d.csv

In [14]:
def regression_metrics(data):
    n=len(data)
    data['ei']= data.apply(lambda x: abs(x['y'] - x['pred']), axis=1) 
    data['mse']= data['ei'].apply(lambda x: x*x)
    total=data['mse'].sum()
    mse=total/n
    mape=(data['ei'].sum())/(data['y'].sum())
    mean=(data['y'].sum())/n 
    ssres=data['mse'].sum()
    data['sstotal']= data.apply(lambda x: (x['y'] - mean), axis=1)
    data['sstotal']= data['sstotal'].apply(lambda x: x*x)
    sstotal=data['sstotal'].sum()
    rsquared=1-(ssres/sstotal)
    return mse,mape,rsquared

In [15]:
mse,mape,rsquared=regression_metrics(df_d)
print('Mean Squared  Error -->',mse)
print('Mean Absolute % Error -->',mape*100)
print('R Squared  -->',rsquared)

Mean Squared  Error --> 177.16569974554707
Mean Absolute % Error --> 12.91202994009687
R Squared  --> 0.9563582786990937


In [16]:
# reference --> Applied AI Videos  + Internet( ML Sites )